In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from statsmodels.formula.api import ols
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

induced_data = pd.read_csv('induced_genes_outliers_removed_all_zero_removed.csv')
induced_data = induced_data.iloc[:,1:]

#grp1 = ['IFITM3','SRGN','IL1RN','IFI27L2A','TNF','IFI205','TARM1','TNFSF9','IRG1','GLIPR2','INHBA','BTG1','PSME2','CLEC4E','2310016C08RIK','SQSTM1','NFKBIA','USP18','EBI3','CD86','MS4A6D','NT5C3','FPR2','H3F3B','SERPINA3G','CAV1','FAS','MS4A6C','LGALS9','GM4902','CD200','IFI204','CISH','PROCR','SLAMF7','AI607873','SGK1','IL15','ACPP','CXCL16','SLFN2','H2-Q7','MT2','H2-T23','THBS1','UBE2L6','AY096003','H1F0','ATF3','TANK','TNFSF15','HSPA5','IL15RA','IRF1','MED21','IRGM1','CCRL2','ZBP1','CASP1','RAB32','SP100','IRF8','BLNK','PSMB9','PCGF5','KTELC1','RILPL2','TMEM39A','TRIM30A','TNIP3','RBM7','SLC2A6','GBP3','BIRC3','TLR2','GTF2B','CFB','SBDS','CAR13','ARF4','SEPW1','BST2','NMI','DUSP1','PYHIN1','CD47','ZFP36','CD274','SAMHD1','SWAP70','IL7R','RELA','AK042010','A130040M12RIK','D14ERTD668E','NFKBIZ','ADAP2','XAF1','PNP','ZUFSP','IFI35','ACSL1','ANXA7','SLC7A2','HERC6','TAPBP','SLFN5','SLFN8','IFIH1','FGL2','LCP2','STAT1','GBP5','TRIM30D','LAP3','STAT5A','GBP6','DDX58','CLIC4','DHX58','STAT2','TNFAIP3','TAP1','ELL2','PIK3R5','PARP9','KPNA3','HIF1A','AFTPH','DTX3L']
#grp2 = ['SDC4','CXCL1','IL1A','CXCL3','IL1B','CCL3','AW112010','CCL17','PRDX1','CCL4','H2-M2']
#with open("global_regressions/summary.csv","w") as logfile:
#    logfile.write("gene,scoreAll\n")


#glist=['CCL5','CCL17','AK217941','AK138792','PRDX1','CXCL10','APOL7C','AW112010','IL1B','CXCL1','CCL3','SDC4','H2-M2','LY6C2','CLU','IL1A','ZDHHC21','CXCL3','CCR7','CD14','CD69','BCL2A1B','IFI27L2A','SLPI','SERPINB9','IFITM3','AA467197','GLIPR2','MX1','TNF','TARM1','BCL2A1D','CLEC4D','IRG1','CD40','BCL2A1A','USP18','SAMSN1','CLEC4E','CAR4','1810029B16RIK','CD86','SQSTM1','SERPINB6B','RNASET2B','MS4A6D','NUPR1','P2RY14','2310016C08RIK','H2-Q7','FOS','GBP2','CDKN1A','NFKBIA','FPR1','EBI3','FPR2','SAMHD1','UBE2L6','IFIT2','MS4A6C','ARF4','STXBP3A','AY096003','EPSTI1','TNIP3','AI607873','SLAMF7','CD200','RAB8B','MT2','SLFN2','HSPA5','BIRC3','IRGM1','MARCKSL1','TMEM39A','ETV3','TREX1','PROCR','BLNK','RELA','RTP4','CCRL2','TRIM30A','BHLHE40','PLAUR','KYNU','TLR2','BST2','SWAP70','ZBP1','SP100','GBP3','PCGF5','KTELC1','IRF8','IGTP','SOD2','CISH','TNFSF15','MNDAL','CFB','PLEK','PNP','ZFP36','NMI','SLC2A6','H1F0','SBDS','XAF1','NFKBIZ','TRAF1','PYHIN1','MTPN','SLC7A2','APOBEC3','CD274','TAPBP','LGALS3BP','KTN1','PPP1CB','ZUFSP','PTTG1','IFIH1','GM4951','BATF','ADAP2','IFI203','IFI35','GADD45B','PELI1','STAT1','LCP2','STAT2','GBP6','KPNA3','SLFN5','GBP5','PFKP','TAP1','TOR3A','INSIG1','DAB2','AFTPH','TXNRD1']

regression_data=pd.read_csv('summary_mean_var_all_cond.csv')
glist=regression_data.iloc[:,0].tolist();


def fit_func(x, a, b):
    return b-a*np.log(x)

In [3]:
#print(induced_data.head)

genelist = induced_data.iloc[:,0].reset_index(drop=True).drop_duplicates().tolist()

i=0

plt.subplots(1,1,figsize=(5,4.5))
axes = plt.gca()
axes.set_xlim([0,3000])
axes.set_ylim([0,10000])

with open("my_summary_mean_burst_all_conds_log.csv","a") as logfile:
        logfile.write("%s,%s,%s,%s,%s,%s,%s,\n" % ('gene','N','intercept','slope','min x','max x','R2'))

for test_gene in glist:
    i+=1
    print(i)
    summary_stats = induced_data[induced_data["Gene"]==test_gene]
    #print(summary_stats)
    if (len(summary_stats.loc[summary_stats['Mean'] == 0,['Mean']])>0) :
        print("Missing data for %s" % test_gene)
        with open("errorlog.csv","a") as logfile:
            logfile.write("%s\n" % test_gene)
        continue
    
    print("Working on %s mean/burst regression..." % test_gene)
    
    x=np.ravel(summary_stats['Mean'].values.reshape(-1, 1))
    #print(x)
    y=np.ravel(summary_stats['Burst Size'].values.reshape(-1, 1))
    #print(y)
    params = curve_fit(fit_func, x, y)
    [a, b] = params[0]
    
    scoreAll=r2_score(y,b-a*np.log(x))
    print("R^2 score:\t",scoreAll)

    linecol = 'gray'
    
    #if test_gene in glist:
    plt.plot(sorted(x),b-a*np.log(sorted(x)), color=linecol, linewidth=1)

    with open("my_summary_mean_burst_all_conds_log.csv","a") as logfile:
        logfile.write("%s,%s,%s,%s,%s,%s,%s,\n" % (test_gene,len(summary_stats['Mean']),b,a,min(summary_stats['Mean']), max(summary_stats['Mean']), scoreAll))
        print("%s,%s\n" % (test_gene,scoreAll))

plt.ylabel("Burst")
plt.xlabel("Mean")
#plt.savefig("global_regressions/mean_var_composite.png")
plt.savefig("my_mean_burst_all_conds_log.png")
plt.close()




1
Working on CXCL1 mean/burst regression...
R^2 score:	 0.665998618403
CXCL1,0.665998618403

2
Working on RSAD2 mean/burst regression...
R^2 score:	 0.804631372439
RSAD2,0.804631372439

3
Working on OASL1 mean/burst regression...
R^2 score:	 0.791911164641
OASL1,0.791911164641

4
Working on NT5C3 mean/burst regression...
R^2 score:	 0.175085787667
NT5C3,0.175085787667

5
Working on CCL4 mean/burst regression...
R^2 score:	 0.574814146783
CCL4,0.574814146783

6
Working on EBI3 mean/burst regression...
R^2 score:	 0.540645528022
EBI3,0.540645528022

7
Working on IRGM1 mean/burst regression...
R^2 score:	 0.275326275543
IRGM1,0.275326275543

8
Working on IFIT2 mean/burst regression...
R^2 score:	 0.791583681492
IFIT2,0.791583681492

9
Working on PYHIN1 mean/burst regression...
R^2 score:	 0.133160218096
PYHIN1,0.133160218096

10
Working on CAR4 mean/burst regression...
R^2 score:	 0.489899996415
CAR4,0.489899996415

11
Working on SLFN8 mean/burst regression...
R^2 score:	 0.506124677852
S

R^2 score:	 0.374044927899
IFI27L2A,0.374044927899

115
Working on IFI47 mean/burst regression...
R^2 score:	 0.574695713688
IFI47,0.574695713688

116
Working on CLU mean/burst regression...
R^2 score:	 0.664105090691
CLU,0.664105090691

117
Working on 2310016C08RIK mean/burst regression...
R^2 score:	 0.20145721966
2310016C08RIK,0.20145721966

118
Working on KPNA3 mean/burst regression...
R^2 score:	 0.272373303653
KPNA3,0.272373303653

119
Working on GLIPR2 mean/burst regression...
R^2 score:	 0.35771668657
GLIPR2,0.35771668657

120
Working on XAF1 mean/burst regression...
R^2 score:	 0.335245212027
XAF1,0.335245212027

121
Working on SLC7A2 mean/burst regression...
R^2 score:	 0.516512744356
SLC7A2,0.516512744356

122
Working on PROCR mean/burst regression...
R^2 score:	 0.228925319698
PROCR,0.228925319698

123
Working on PRPF38A mean/burst regression...
R^2 score:	 0.529887881606
PRPF38A,0.529887881606

124
Working on 1810029B16RIK mean/burst regression...
R^2 score:	 0.40573304290

R^2 score:	 0.0231403496779
H2-M2,0.0231403496779

265
Working on BATF mean/burst regression...
R^2 score:	 0.196566363957
BATF,0.196566363957

266
Working on NUP62-IL4I1 mean/burst regression...
R^2 score:	 0.145467834179
NUP62-IL4I1,0.145467834179

267
Working on TNFAIP2 mean/burst regression...
R^2 score:	 0.0184979042588
TNFAIP2,0.0184979042588

268
Working on FNDC3A mean/burst regression...
R^2 score:	 0.232854815209
FNDC3A,0.232854815209

269
Working on TREX1 mean/burst regression...
R^2 score:	 0.0403319871384
TREX1,0.0403319871384

270
Working on RAB9 mean/burst regression...
R^2 score:	 0.317414319966
RAB9,0.317414319966

271
Working on BCL2A1A mean/burst regression...
R^2 score:	 0.0550465683896
BCL2A1A,0.0550465683896

272
Working on PDE4B mean/burst regression...
R^2 score:	 0.0554932351945
PDE4B,0.0554932351945

273
Working on PIK3R5 mean/burst regression...
R^2 score:	 0.188260071739
PIK3R5,0.188260071739

274
Working on RND3 mean/burst regression...
R^2 score:	 0.1417479